# AHU Components

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

Import sales file

In [2]:
sales_file = 'Sales.xlsx'
cols = ['Order no', 'Line no', 'Item no', 'Amount', 'Cost amount', 'Ref order no']
df_sales = pd.read_excel(sales_file, usecols = cols)

# Check formats
for col in cols:
    df_sales[col] = df_sales[col].astype(str)

# Rename columns
df_sales = df_sales.rename(columns={'Ref order no': 'MO no'})
df_sales.head()

,Order no,Line no,Item no,Amount,Cost amount,MO no
0,3899063,1,BNBRV,0.0,18449.79,957088
1,4395494,1,B70107,0.0,0.0,2096185
2,4519553,1,E5019,844.0,717.4,
3,4597123,1,B70102,0.0,5411.2,
4,4728958,1,27954,8303.0,6910.79,


Import MO's file for the 645-100 work-center and filtered by the period to study (natural year for example)

In [3]:
mo_file = '645-100.xlsx'
cols = ['MO no', 'Component no']
df_mo = pd.read_excel(mo_file, usecols = cols)

# Check formats
for col in cols:
    df_mo[col] = df_mo[col].astype(str)
    
df_mo.head()

,MO no,Component no
0,1005210962,90925520
1,1005210962,96815
2,1005210962,90812006
3,1005210962,90715005
4,1005210962,89129


Import components file - This data needs to be checked periodically in order to have better stats

In [4]:
comp_file = 'Components.xlsx'
df_comp = pd.read_excel(comp_file)
df_comp['Component no'] = df_comp['Component no'].astype(str)
df_comp.head()

,Component no,Description,Component
0,90732769,"ZAbluefin-25-0,50kW",EC-fan
1,2685841,GR28C 0.48kW 2510rpm,EC-fan
2,90732650,GR25C 0.50kW 3080rpm,EC-fan
3,90732770,"ZAbluefin-25-0,78kW",EC-fan
4,2685830,GR25C 0.50kW 3080rpm,EC-fan


Merge MO with Component

In [5]:
df_map = pd.merge(df_mo, df_comp, on='Component no')
df_map.head()

,MO no,Component no,Description,Component
0,1005210962,89129,18-F9-520,Filters
1,1005409482,89129,18-F9-520,Filters
2,1005506688,89129,18-F9-520,Filters
3,1005506746,89129,18-F9-520,Filters
4,1005105591,89129,18-F9-520,Filters


Get pivot

In [6]:
df_pivot = pd.pivot_table(df_map, values = 'Component no', index = 'MO no',
                          columns = 'Component', aggfunc = 'count')
df_pivot = pd.DataFrame(df_pivot.to_records())

# Create only one check for each component
div = df_pivot.iloc[:, 1:]/df_pivot.iloc[:, 1:]
df_pivot = pd.concat([df_pivot['MO no'], div], axis = 1)
df_pivot = df_pivot.fillna(0)

df_pivot.head()

,MO no,Adiabatic humidifier,Atex,Control on/off adiabatic hum.,Control stages adiabatic hum.,Damper,EC-fan,Electrical heater,Filters,Louver,Metallic roof,Plate HE,Plug-fan,Rotary HE,Sound attenuator,Steam lance,UV lamp
0,1004689962,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1004835641,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1004835746,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1004840141,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1004908120,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


Merge to the final dataframe and remove zero-value rows. Rearrange the columns

In [7]:
df_final = pd.merge(df_sales, df_pivot, on='MO no')
df_final = df_final.fillna(0)
#df_final = df_final[df_final['Order no'] != 0]
df_final.head()

,Order no,Line no,Item no,Amount,Cost amount,MO no,Adiabatic humidifier,Atex,Control on/off adiabatic hum.,Control stages adiabatic hum.,...,Electrical heater,Filters,Louver,Metallic roof,Plate HE,Plug-fan,Rotary HE,Sound attenuator,Steam lance,UV lamp
0,5388305,1,AHU-DV80,16664.69,15945.343,1004835746,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,5438247,1,AHU-GX20C,16411.0,11308.39,1004908120,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5470849,1,AHU-GX24,16079.61,13730.057,1005264166,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,5477812,1,AHU-GX10,3511.0,2667.179,1004970908,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5478895,1,AHU-GX24C,10497.0,6772.429,1004978386,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


Export to Excel

In [8]:
name = 'AHU-Components - Results.xlsx'
writer = pd.ExcelWriter(name)
df_final.to_excel(writer, index = False)
writer.save()